In [1]:
import io
import os
import re
import sys
import csv
import gzip
import subprocess
import pandas as pd
import numpy as np
import itertools
import collections
sys.path.append('/well/band/users/rbx225/software/lcwgsus/')
import lcwgsus
from lcwgsus.variables import *

<Figure size 100x100 with 0 Axes>

In [13]:
hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')

# hlatypes = pd.read_csv('/well/band/users/rbx225/GAMCC/results/hla/imputation/ref_panel/auxiliary_files/20181129_HLA_types_full_1000_Genomes_Project_panel.txt', sep = '\t')
samples = lcwgsus.read_tsv_as_lst('/well/band/users/rbx225/recyclable_files/ref_panels/oneKG_30x/samples_to_phase.tsv')
# hlatypes = hlatypes[hlatypes['Sample ID'].isin(samples)].sort_values(by = 'Sample ID').reset_index(drop = True)
# for c in hlatypes.columns[3:]:
#     hlatypes[c] = hlatypes[c].astype(str)
#     hlatypes[c] = hlatypes[c].str.replace('*', '', regex=False)
#     hlatypes[c] = hlatypes[c].apply(lambda x: 'N/A' if '/' in x else x)
# hlatypes.to_csv('/well/band/users/rbx225/GAMCC/results/hla/imputation/ref_panel/auxiliary_files/HLA_types_2568_for_phasing.txt', sep = '\t', header = True, index = False)

# The new hlatypes file 1) has all asterisks removed 2) ambiguous types set to N/A 3) only 2568 samples

hlatypes = pd.read_csv('/well/band/users/rbx225/GAMCC/results/hla/imputation/ref_panel/auxiliary_files/HLA_types_2568_for_phasing.txt', sep = '\t')
phased_vcf = "/well/band/users/rbx225/recyclable_files/ref_panels/oneKG_30x/oneKG.chr6.vcf.gz"

start = 25000000
end = 34000000

command = f"bcftools view -s {','.join(samples)} -r chr6:" + str(int(start)) + "-" + str(int(end)) + " " + phased_vcf
vcf = subprocess.run(command, shell = True, capture_output = True, text = True).stdout[:-1].split('\n')
vcf = [i.split('\t') for i in vcf if '##' not in i]
vcf = pd.DataFrame(vcf)
vcf.columns = vcf.iloc[0]  # Set the first row as the header
vcf = vcf[1:].reset_index(drop = True)
vcf['POS'] = vcf['POS'].astype(int)

distinct_alleles = {}
for g in HLA_GENES:
    alleles = np.append(hlatypes[f'HLA-{g} 1'].astype(str).to_numpy(), hlatypes[f'HLA-{g} 2'].astype(str).to_numpy())
    alleles = np.unique(alleles)
    alleles = alleles[alleles != 'nan']
    distinct_alleles[g] = alleles

test = vcf.copy()

for g in HLA_GENES:
    r = hla_gene_information[hla_gene_information['Name'] == f'HLA-{g}']
    start = r['Start'].values[0]
    all_snp_pos = vcf['POS'].tolist()
    position = start
    n_alleles = 0

    while n_alleles < len(distinct_alleles[g]):
        if position in all_snp_pos:
            pass
        else:
            twofield = distinct_alleles[g][n_alleles]
            common_cols = ['chr6', position, f'HLA_{g}*{twofield}', 'T', 'A', '.', 'PASS', '.', 'GT']
            allele1 = (hlatypes[f'HLA-{g} 1'] == twofield).astype(int).values
            allele2 = (hlatypes[f'HLA-{g} 2'] == twofield).astype(int).values

            gts = [f'{a1}/{a2}' for a1, a2 in zip(allele1, allele2)]
            vcf.loc[len(vcf)] = common_cols + gts
            n_alleles += 1
        position += 1

vcf = vcf.sort_values(by = 'POS', ascending = True).reset_index(drop = True)

# metadata = lcwgsus.read_metadata(phased_vcf, filetype = 'gzip', comment = '#', new_cols = None)
# metadata[-1] = '\t'.join(vcf.columns)
# lcwgsus.save_vcf(vcf,
#              metadata,
#              rezip = True,
#              prefix='chr',
#              outdir='/well/band/users/rbx225/GAMCC/results/phasing/HLA_1KG_BEAGLE/',
#              save_name='unphased.1KG.chr6.vcf.gz'
#              )

,Region,Population,Sample ID,HLA-A 1,HLA-A 2,HLA-B 1,HLA-B 2,HLA-C 1,HLA-C 2,HLA-DQB1 1,HLA-DQB1 2,HLA-DRB1 1,HLA-DRB1 2
0,EUR,GBR,HG00096,01:01,29:02,08:01,44:03,07:01,16:01,02:01,02:02,03:01,07:01
1,EUR,GBR,HG00097,03:01,24:02,07:02,07:02,07:02,07:02,03:01,06:02,13:03,15:01
2,EUR,GBR,HG00099,01:01,68:01,08:01,44:02,07:01,07:04,02:01,03:01,03:01,11:01
3,EUR,GBR,HG00100,01:01,01:01,08:01,57:01,06:02,07:01,02:01,03:03,03:01,07:01
4,EUR,GBR,HG00101,11:01,32:01,27:05,57:01,02:02,06:02,03:02,06:02,04:04,15:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2563,SAS,GIH,NA21137,01:01,11:01,57:01,40:06,06:02,15:02,03:03,06:01,09:01,15:01
2564,SAS,GIH,NA21141,02:11,24:02,15:05,18:01,03:03,07:01,NaN,05:03,07:01,15:02
2565,SAS,GIH,NA21142,03:01,24:02,35:01,44:03,04:01,07:06,05:01,06:09,01:01,13:02
2566,SAS,GIH,NA21143,11:01,26:01,08:01,57:01,06:02,07:02,02:01,03:03,03:01,07:01


In [ ]:
beagle_phasing = "/well/band/users/rbx225/GAMCC/results/phasing/HLA_1KG_BEAGLE/phased.1KG.chr6.vcf.gz"
sus_phasing = "/well/band/users/rbx225/GAMCC/results/hla_ref_panel/oneKG_mGenv1/oneKG_mGenv1_HLA_calls.tsv"